In [ ]:
import pandas as pd
import os
import altair as alt
import folium
import matplotlib.pyplot as plt

In [ ]:
# SN changes
# add month
# remove .isocalendar()
# filter to 2017 and after

home_value_df = pd.read_csv(os.path.join('..','data','cleaned_data','ny_zhvi.csv'))
home_value_df.drop(['RegionType','StateName','State','City','Metro','CountyName'], inplace=True, axis=1) 
home_value_df.rename({'RegionName': 'zipcode'}, inplace=True, axis=1)
home_value_df.set_index('zipcode',inplace=True)
home_value_df_transposed = home_value_df.T
home_value_df_transposed.reset_index(inplace=True)
cols = [str(a) for a in list(home_value_df_transposed)]
cols[0] = 'date_str'
home_value_df_transposed.columns = cols


home_value_df_transposed_slim = pd.DataFrame(columns=['date_str', 'mean_zhvi',  'zipcode'])

for zip_ in list(home_value_df_transposed):
    if zip_ == 'date_str':
        continue
    current_zip = home_value_df_transposed[['date_str', zip_]].copy()
    current_zip['zipcode'] = zip_
    current_zip.columns = ['date_str', 'mean_zhvi',  'zipcode']
    home_value_df_transposed_slim = home_value_df_transposed_slim.append(current_zip)
home_value_df_transposed_slim['mean_zhvi'] = home_value_df_transposed_slim['mean_zhvi'].astype(float)

home_value_df_transposed_slim['date'] = pd.to_datetime(home_value_df_transposed_slim['date_str'])
home_value_df_transposed_slim['date_month'] = home_value_df_transposed_slim['date'].dt.month
home_value_df_transposed_slim['date_year'] = home_value_df_transposed_slim['date'].dt.year
zhvi_grouped = home_value_df_transposed_slim.groupby(['zipcode', 'date_month','date_year'], as_index=False).agg({'mean_zhvi':'mean'})
zhvi_grouped = zhvi_grouped[zhvi_grouped.date_year >= 2017]

In [ ]:
crime_2017_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2017.csv'))
crime_2018_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2018.csv'))
crime_2019_df = pd.read_csv(os.path.join('..', 'data', 'cleaned_data', 'ny_crimes_2019.csv'))

print(f'2017 crime df shape: {crime_2017_df.shape}')
print(f'2018 crime df shape: {crime_2018_df.shape}')
print(f'2019 crime df shape: {crime_2019_df.shape}')

In [ ]:
all_crimes_full_df = crime_2017_df.append(crime_2018_df)
all_crimes_full_df = all_crimes_full_df.append(crime_2019_df)
print(all_crimes_full_df.shape)

In [ ]:
all_crimes_full_df['ARREST_DATE_STR'] = all_crimes_full_df['ARREST_DATE'].copy()
all_crimes_full_df['ARREST_DATE'] = pd.to_datetime(all_crimes_full_df['ARREST_DATE_STR']).copy()
all_crimes_full_df['ARREST_DATE_MONTH'] = all_crimes_full_df['ARREST_DATE'].dt.month
all_crimes_full_df['ARREST_DATE_YEAR'] = all_crimes_full_df['ARREST_DATE'].dt.year
all_crimes_full_df['zip_code'] = all_crimes_full_df['zip_code'].astype(str)

In [ ]:
# SN changes
# add month
crime_by_zip = all_crimes_full_df.groupby(['zip_code', 'ARREST_DATE_MONTH','ARREST_DATE_YEAR'], as_index=False).agg({'ARREST_KEY': 'count'})
crime_by_zip = crime_by_zip[(crime_by_zip.ARREST_DATE_YEAR > 2016) & (crime_by_zip.ARREST_DATE_YEAR < 2020)]
crime_by_zip.columns = ['zipcode','month','year','total_arrest_count']
crime_by_zip.shape

In [ ]:
all_crimes_full_df.head()

In [ ]:
crime_by_zip_m = all_crimes_full_df[all_crimes_full_df.LAW_CAT_CD == 'M'].groupby(['zip_code', 'ARREST_DATE_MONTH','ARREST_DATE_YEAR'], as_index=False).agg({'ARREST_KEY': 'count'})
crime_by_zip_m = crime_by_zip_m[(crime_by_zip_m.ARREST_DATE_YEAR > 2016) & 
                                (crime_by_zip_m.ARREST_DATE_YEAR < 2020)]
crime_by_zip_m.columns = ['zipcode','month','year','total_arrest_count']
crime_by_zip_m.shape

In [ ]:
crime_by_zip_f = all_crimes_full_df[all_crimes_full_df.LAW_CAT_CD == 'F'].groupby(['zip_code', 'ARREST_DATE_MONTH','ARREST_DATE_YEAR'], as_index=False).agg({'ARREST_KEY': 'count'})
crime_by_zip_f = crime_by_zip_f[(crime_by_zip_f.ARREST_DATE_YEAR > 2016) & 
                                (crime_by_zip_f.ARREST_DATE_YEAR < 2020)]
crime_by_zip_f.columns = ['zipcode','month','year','total_arrest_count']
crime_by_zip_f.shape

In [ ]:
crime_by_zip.head(20)

In [ ]:
zhvi_grouped_filtered = zhvi_grouped[(zhvi_grouped.date_year > 2016) & (zhvi_grouped.date_year < 2020)].copy()
print(zhvi_grouped_filtered.shape)

In [ ]:
crime_by_zip[(crime_by_zip.month==1) &
                      (crime_by_zip.year == 2017)]['total_arrest_count'].sum()

In [ ]:
crime_by_zip.groupby(['month', 'year'], as_index=False).agg({'total_arrest_count': 'sum'})

In [ ]:
# SN change
# add month to merge condition
# change join to left and fill missing total_arrest_count with 0
# add month year
full_df = zhvi_grouped_filtered.merge(crime_by_zip_f, left_on=['zipcode','date_month','date_year'], right_on=['zipcode','month','year'], how='left')
full_df["total_arrest_count"] = full_df["total_arrest_count"].fillna(0)
full_df["month"] = full_df["date_month"]
full_df["year"] = full_df["date_year"]
full_df['month_year'] = full_df['month'].map(str)+ '-' +full_df['year'].map(str)
full_df['month_year'] = pd.to_datetime(full_df['month_year'], format='%m-%Y').dt.strftime('%m-%Y')
full_df.drop(['date_month','date_year'], inplace=True, axis=1)
full_df.head()

In [ ]:
# full_df.groupby(['month','year'], as_index=False).agg({'total_arrest_count': 'sum', 'mean_zhvi': 'mean'}).sort_values(['year','month'])

In [ ]:
# correct

# by month all NY aggregated

full_df_summary = full_df.groupby(['month','year'], as_index=False).agg({'total_arrest_count': 'sum', 'mean_zhvi': 'mean'})
full_df_summary['mean_zhvi'].corr(full_df_summary['total_arrest_count'])

In [ ]:
# by year, all NY aggregated
full_df_summary_test = full_df.groupby(['year'], as_index=False).agg({'total_arrest_count': 'sum', 'mean_zhvi': 'mean'})
full_df_summary_test['mean_zhvi'].corr(full_df_summary_test['total_arrest_count'])

In [ ]:
# by month all NY NOT aggregated (by zip)
full_df['mean_zhvi'].corr(full_df['total_arrest_count'])

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np

# get linear model coefficients and slope
linear_full_df = full_df_summary.copy()
linear_full_df.fillna(method='bfill', inplace=True)

linear_model = LinearRegression()
linear_model.fit(np.array(linear_full_df['total_arrest_count']).reshape(-1, 1), np.array(linear_full_df['mean_zhvi']).reshape(-1, 1))

In [ ]:
arrest_list = list(np.linspace(start=full_df_summary['total_arrest_count'].min(), stop=full_df_summary['total_arrest_count'].max()*1.05, num=1000))
zhvi_list = [((a*linear_model.coef_) + linear_model.intercept_).item() for a in arrest_list]
linear_output_df = pd.DataFrame({'zhvi_list': zhvi_list, 'arrest_list': arrest_list})
linear_output_df.head()

In [ ]:
c1 = alt.Chart(full_df_summary).mark_point().encode(
    x=alt.X('total_arrest_count', scale=alt.Scale(domain=[12000, 26000]), axis=alt.Axis(title='Total Arrests')),
    y=alt.Y('mean_zhvi', scale=alt.Scale(domain=[680000, 730000]), axis=alt.Axis(title='Mean ZHVI ($)')),
    color='year:N')

c2 = alt.Chart(linear_output_df).mark_line(color='green').encode(
    y=alt.Y('zhvi_list'),
    x=alt.X('arrest_list'))

(c1 + c2).properties(title='Correlation Between Total Arrests and Mean ZHVI')

In [ ]:
# residuals

preds = linear_model.predict(np.array(linear_full_df['total_arrest_count']).reshape(-1, 1))
actuals = np.array(linear_full_df['mean_zhvi']).reshape(-1, 1)

# on average, predictions using only crime rate per capita to predict ZHVI will be off by $250k
(np.abs(actuals - preds)).mean()

In [ ]:
from sklearn.metrics import r2_score

print(f'r-squared of fitted model: {r2_score(actuals, preds):.3f}')

#### Correlation to Per Capita Arrests by Zip

In [ ]:
pop_by_zip_df = pd.read_csv(os.path.join('..','data','cleaned_data','population_by_zip.csv'))
pop_by_zip_df.columns=['zipcode','population']
pop_by_zip_df['population_int'] = pop_by_zip_df['population'].apply(lambda a: int(a.replace(',','')))
pop_by_zip_df.drop(['population'], inplace=True, axis=1)
pop_by_zip_df['zipcode'] = pop_by_zip_df['zipcode'].astype(str).str[:5]
pop_by_zip_df.head()

full_df = full_df.merge(pop_by_zip_df, left_on=['zipcode'], right_on=['zipcode'], how='left')
full_df['arrest_per_capita'] = full_df['total_arrest_count'] / full_df['population_int']
full_df.head()

In [ ]:
full_df['mean_zhvi'].corr(full_df['arrest_per_capita'])

In [ ]:
zip_correlation_dict = {}
for zip in full_df.zipcode.unique():
    zip_df = full_df[full_df.zipcode == zip]
    zip_correlation_dict[zip] = zip_df['mean_zhvi'].corr(zip_df['arrest_per_capita'])
    
zip_correlation_df = pd.DataFrame.from_dict(zip_correlation_dict, orient = 'index').reset_index()
zip_correlation_df.columns = ['zipcode','correlation_value']
zip_correlation_df = zip_correlation_df[~zip_correlation_df.correlation_value.isnull()]
zip_correlation_df.head()

#### Correlation to Arrests by Zip

In [ ]:
zip_correlation_dict = {}
for zip in full_df.zipcode.unique():
    zip_df = full_df[full_df.zipcode == zip]
#     zip_df.sort_values(by = ['zipcode','year','month'], inplace = True)
    zip_correlation_dict[zip] = zip_df['mean_zhvi'].corr(zip_df['total_arrest_count'])

In [ ]:
zip_correlation_df = pd.DataFrame.from_dict(zip_correlation_dict, orient = 'index').reset_index()
zip_correlation_df.columns = ['zipcode','correlation_value']
zip_correlation_df = zip_correlation_df[~zip_correlation_df.correlation_value.isnull()]
zip_correlation_df.head()

In [ ]:
print(zip_correlation_df[(zip_correlation_df['correlation_value'] > 0.5)].shape)
print(zip_correlation_df[(zip_correlation_df['correlation_value'] < -0.5)].shape)

In [ ]:
# full_df[full_df.zipcode=='10473'].groupby(['month','year'], as_index=False).agg({'total_arrest_count': 'sum', 
#                                                                                  'mean_zhvi': 'mean'}).sort_values(['year','month'])

In [ ]:
# average crime per zip?

# full_df_summary = full_df[['month_year','mean_zhvi','total_arrest_count']].groupby('month_year').mean().reset_index()
# full_df_summary.sort_values(by = ['month_year'], inplace = True)
# full_df_summary['mean_zhvi'].corr(full_df_summary['total_arrest_count'])
# full_df_summary

In [ ]:
zip_correlation_dict = {}
for zip in full_df.zipcode.unique():
    zip_df = full_df[full_df.zipcode == zip]
    zip_df.sort_values(by = ['zipcode','year','month'], inplace = True)
    zip_correlation_dict[zip] = zip_df['mean_zhvi'].corr(zip_df['total_arrest_count'])

In [ ]:
zip_correlation_df = pd.DataFrame.from_dict(zip_correlation_dict, orient = 'index').reset_index()
zip_correlation_df.columns = ['zipcode','correlation_value']
zip_correlation_df = zip_correlation_df[~zip_correlation_df.correlation_value.isnull()]
zip_correlation_df.sort_values('correlation_value').head()

In [ ]:
# nyc_map = folium.Map(location=[40.693943, -73.985880], zoom_start=10, tiles = "CartoDB positron")
# folium.Choropleth(
#     geo_data = "nyc-zip-codes.geojson", #json
#     name ='choropleth',                  
#     data = zip_avg_zhvi,                     
#     columns = ['zipcode', 'mean_zhvi'], #columns to work on
#     key_on ='feature.properties.postalCode',
#     fill_color ='YlGnBu',     #I passed colors Yellow,Green,Blue
#     fill_opacity = 0.7,
#     line_opacity = 0.2,
#    legend_name = "Mean ZHVI (Housing Prices)"
# ).add_to(nyc_map)

# nyc_map

In [ ]:
nyc_map = folium.Map(location=[40.693943, -73.985880], zoom_start=10,tiles="CartoDB positron")
folium.Choropleth(
    geo_data = "nyc-zip-codes.geojson", #json
    name ='choropleth',                  
    data = zip_correlation_df,                     
    columns = ['zipcode', 'correlation_value'], #columns to work on
    key_on ='feature.properties.postalCode',
    fill_color ='YlGnBu',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
   legend_name = "Correlation Between Number of Arrests and Housing Prices"
).add_to(nyc_map)

nyc_map

In [ ]:
zip_correlation_df[zip_correlation_df.correlation_value > 0.2]

In [ ]:
# average zhvi and total arrest across all zips for each month and plot
full_df_summary = full_df[['month_year','mean_zhvi','total_arrest_count']].groupby('month_year').mean().reset_index()
full_df_summary.sort_values(by = ['month_year'], inplace = True)
print('Overall Correlation of Housing Prices vs. Total Arrest Count in NYC')
print(full_df_summary['mean_zhvi'].corr(full_df_summary['total_arrest_count']))

import matplotlib.pyplot as plt

fig, ax1 = plt.subplots()
ax1.plot_date(full_df_summary['month_year'], tst_df['mean_zhvi'], c = 'b')
ax1.set_ylabel('Mean of Housing Prices')
ax2 = ax1.twinx()
ax2.plot_date(full_df_summary['month_year'], tst_df['total_arrest_count'], c = 'r')
ax2.set_ylabel('Mean of Total Arrest Counts')
ax2.set_xticks(ax1.get_xticks()[::10])
plt.show()

In [ ]:
# plot the total_arrests vs. housing prices for zip code with correlation greater than 0.2
import numpy as np
fig, axs = plt.subplots(6,3, figsize=(18,12))
axs = axs.ravel()

i = 0
for zip in zip_correlation_df[zip_correlation_df.correlation_value > 0.2].zipcode:
    tst_df = full_df[full_df.zipcode == zip].sort_values(['year','month'])
    axs[i].plot_date(tst_df['month_year'], tst_df['mean_zhvi'], c = 'b')
    axs[i].twinx().plot_date(tst_df['month_year'], tst_df['total_arrest_count'], c = 'r')
    axs[i].set_xticks(ax1.get_xticks()[::2])
    axs[i].set_title("Zip: " + str(zip))
    i = i + 1
    
fig.suptitle('Housing Prices vs. Total Arrest Count for Zips With Correlation > 0.2', fontsize=16)
fig.tight_layout()